In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!git clone https://github.com/peppermintcoding/Calliope.git
!pip install -U -q autoawq accelerate openai

In [ ]:
from transformers import AwqConfig, AutoModelForCausalLM, AutoTokenizer
import time

# model_name_or_path = "TheBloke/UNA-TheBeagle-7B-v1-AWQ"
model_name_or_path = "TheBloke/OpenHermes-2.5-Mistral-7B-AWQ"
# model_name_or_path = "TheBloke/Nous-Hermes-2-SOLAR-10.7B-AWQ"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    low_cpu_mem_usage=True,
    device_map="cuda")
model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
from Calliope.data.llm_poetry import FAMOUS_POETS, POEM_TOPICS, THINGS
import numpy as np

def create_random_prompt():
    poet = np.random.choice(FAMOUS_POETS)
    topic = np.random.choice(POEM_TOPICS)
    thing = np.random.choice(THINGS, size=2, replace=False)

    system_prompt = f"You are a poet like {poet}. You are a very skilled poet, fun, whymsical, extraordinary gifted."
    user_prompt = f"Write a poem about {topic} and incorporate {thing[0]} and {thing[1]} into it."
    return (
        "<|im_start|>system\n"
        f"{system_prompt}<|im_end|>\n"
        "<|im_start|>user\n"
        f"{user_prompt}<|im_end|>\n"
        "<|im_start|>assistant\n"
    )

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

In [ ]:
import pandas as pd
from tqdm import tqdm

poems = []
pbar = tqdm(range(1, 100))
for i in pbar:
  prompts = [create_random_prompt() for _ in range(35)]
  tokens = tokenizer(prompts, return_tensors="pt", padding=True).to("cuda")

  start_time = time.time()
  generation_output = model.generate(**tokens, do_sample=True, temperature=0.95, max_length=2048)
  num_tokens = 0
  for j, output in enumerate(generation_output):
    num_tokens += len(output[len(tokens[j]):])
    poems.append([model_name_or_path, prompts[j], tokenizer.decode(output[len(tokens[j]):], skip_special_tokens=True)])
  end_time = time.time()
  pbar.set_description(f"Generating at {num_tokens / (end_time - start_time):.2f} Tokens/s")
  if i % 10 == 0:
    df = pd.DataFrame(data=poems, columns=["model_name", "prompt", "text"])
    df.to_csv(
      "../gdrive/MyDrive/Calliope/llm_poems.csv",
      index=False,
      mode="a",
      header=False,
    )
    poems = []

# add remainder
df = pd.DataFrame(data=poems, columns=["model_name", "prompt", "text"])
df.to_csv(
  "../gdrive/MyDrive/Calliope/llm_poems.csv",
  index=False,
  mode="a",
  header=False,
)

In [ ]:
df = pd.read_csv("../gdrive/MyDrive/Calliope/llm_poems.csv")
print(f"Number of words: {df['text'].apply(lambda x: len(x.split())).sum():,}")